In [0]:
%sql
with warehouses as (
    select *
      from (
        select *,
              row_number() over (partition by warehouse_id order by change_time desc) as rn
          from system.compute.warehouses
        )
     where rn = 1
),   query_counts as (
    select date_trunc('hour', h.end_time) as end_time_hour,
          h.compute.warehouse_id          as warehouse_id,
          w.warehouse_name                as warehouse_name,
          count(1)                        as query_count
      from system.query.history h
      left join warehouses w
        on h.compute.warehouse_id = w.warehouse_id
     where h.end_time >= current_timestamp() - interval 1 day
       and h.compute.warehouse_id is not null
       and h.workspace_id = '1444828305810485'
     group by h.compute.warehouse_id, w.warehouse_name, date_trunc('hour', h.end_time)
)
select end_time_hour as end_time,
       warehouse_name || ' (' || warehouse_id || ')' as warehouse,
       query_count
  from query_counts

In [0]:
%sql
select *
  from (
    select *,
           row_number() over (partition by warehouse_id order by change_time desc) as rn
      from system.compute.warehouses
  )
where rn = 1

In [0]:
%sql
select h.*
 from system.query.history h
where h.end_time >= current_timestamp() - interval 1 day
  and h.compute.warehouse_id is not null
  and h.workspace_id = '1444828305810485'
  and h.written_bytes > 1024 * 1024 * 100
order by end_time desc

In [0]:
%sql
with warehouses as (
    select *
      from (
        select *,
              row_number() over (partition by warehouse_id order by change_time desc) as rn
          from system.compute.warehouses
        )
     where rn = 1
),   written as (
    select date_trunc('hour', h.end_time) as end_time_hour,
          h.compute.warehouse_id          as warehouse_id,
          w.warehouse_name                as warehouse_name,
          sum(h.written_bytes)            as written_bytes
      from system.query.history h
      left join warehouses w                        
        on h.compute.warehouse_id = w.warehouse_id
     where h.end_time >= current_timestamp() - interval 1 day
       and h.compute.warehouse_id is not null
       and h.workspace_id = '1444828305810485'
     group by h.compute.warehouse_id, w.warehouse_name, date_trunc('hour', h.end_time)
)
select end_time_hour as end_time,
       warehouse_name || ' (' || warehouse_id || ')' as warehouse,
       written_bytes / 1024 / 1024 / 1024 / 1024 as written_gb
  from written
where written_bytes > 0

In [0]:
%sql
select *
  from system.compute.warehouse_events
where warehouse_id = '581ed75401284b97'
  and event_time >= current_timestamp() - interval 7 days
order by event_time desc

In [0]:
%sql
select 
  date_trunc('minute', event_time) - 
    make_interval(
      0, 0, 0, 0, 0, 
      extract(minute from event_time) % 10, 
      0
    ) as interval_start,
  max(cluster_count) as clusters_running
from system.compute.warehouse_events
where warehouse_id = '4b9b953939869799'
  and event_time >= current_timestamp() - interval 7 days
group by 
  date_trunc('minute', event_time) - 
    make_interval(
      0, 0, 0, 0, 0, 
      extract(minute from event_time) % 10, 
      0
    )
order by interval_start